In [1]:
import h5py
import tensorflow as tf
import os
import numpy as np
import json
import shutil

2024-09-22 07:17:23.969107: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-22 07:17:24.153457: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-22 07:17:24.153558: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-22 07:17:24.175929: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-22 07:17:24.234949: I tensorflow/core/platform/cpu_feature_guar

In [2]:
num_episodes = 1000
episodes_per_file = 10
num_files =  num_episodes // episodes_per_file

# input_hdf5_dir = '/media/sbr-tech/Desk SSD/aloha/datas_couple_mobile'
# dataset_name = "mobile_aloha_sim_couple_dataset"
# task_name = "mobile_aloha_sim_couple_dataset"

# input_hdf5_dir = '/media/sbr-tech/Desk SSD1/datas_couple_franka'
# dataset_name = "franka_dual_sim_couple_dataset"
# task_name = "franka_dual_sim_couple_dataset"

input_hdf5_dir = '/media/sbr-tech/Desk SSD/datas_quadruple_franka_bimanual'
# dataset_name = "franka_dual_bimanual_sim_quadruple_dataset"
# task_name = "franka_dual_bimanual_sim_quadruple_dataset"

dataset_name = "franka_dual_bimanual_sim_quadruple_dataset"
task_name = "franka_dual_bimanual_sim_quadruple_dataset"

input_hdf5_basename = "episode_{}.hdf5"
version = "1.0.0"

output_root = "/media/sbr-tech/Desk SSD/"
output_basename = task_name + "-train.tfrecord-{}-of-{}"

mobile = "mobile" in dataset_name
franka = "franka" in dataset_name
bimanual = "bimanual" in dataset_name

lang = "lang" in dataset_name

mobile, franka, bimanual, lang

(False, True)

In [3]:
output_dir = os.path.join(output_root, dataset_name, task_name, version)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [4]:
with open("dataset_info_template.json", "r") as f:
    dataset_info = json.load(f)

dataset_info["moduleName"] = task_name + "." + task_name
dataset_info["name"] = task_name
dataset_info["version"] = version
dataset_info["splits"][0]["shardLengths"] = [str(episodes_per_file)] * num_files 

with open(os.path.join(output_dir, "dataset_info.json"), "w") as f:
    json.dump(dataset_info, f, ensure_ascii=False, indent=2, sort_keys=True, separators=(',', ': '))

In [5]:
# if not mobile:
if not franka:
    ret = shutil.copy("features_template_static.json", os.path.join(output_dir, "features.json"))
else:
    ret = shutil.copy("features_template_franka.json", os.path.join(output_dir, "features.json"))
ret

'/media/sbr-tech/Desk SSD/franka_dual_bimanual_sim_quadruple_dataset/franka_dual_bimanual_sim_quadruple_dataset/1.0.0/features.json'

In [6]:
sample_hdf5_file = os.path.join(input_hdf5_dir, input_hdf5_basename.format(0))

In [7]:
if franka:
    if bimanual:
        NUM_EPISODE = 800 
    else:
        NUM_EPISODE = 720         
elif mobile:
    NUM_EPISODE = 720 
else:
    NUM_EPISODE = 600

IS_FIRSTS = np.zeros(NUM_EPISODE, dtype=int)
IS_FIRSTS[0] = 1
DISCOUNTS = np.ones(NUM_EPISODE, dtype=float)
IS_LASTS = np.zeros(NUM_EPISODE, dtype=int)
IS_LASTS[-1] = 1
LANGUAGE_INSTRUCTION_LEFT_FIRST_CUCUMBER = b"pick up the cucumber with your left hand and the red cube with your right, then put them in the bucket, cucumber first"
LANGUAGE_INSTRUCTION_RIGHT_FIRST_CUCUMBER = b"pick up the cucumber with your right hand and the red cube with your left, then put them in the bucket, cucumber first"
LANGUAGE_INSTRUCTION_LEFT_FIRST_CUBE = b"pick up the cucumber with your right hand and the red cube with your left, then put them in the bucket, cube first"
LANGUAGE_INSTRUCTION_RIGHT_FIRST_CUBE = b"pick up the cucumber with your left hand and the red cube with your right, then put them in the bucket, cube first"
LANGUAGE_INSTRUCTIONS_LEFT_FIRST_CUCUMBER = np.array([LANGUAGE_INSTRUCTION_LEFT_FIRST_CUCUMBER]*NUM_EPISODE)
LANGUAGE_INSTRUCTIONS_RIGHT_FIRST_CUCUMBER = np.array([LANGUAGE_INSTRUCTION_RIGHT_FIRST_CUCUMBER]*NUM_EPISODE)
LANGUAGE_INSTRUCTIONS_LEFT_FIRST_CUBE = np.array([LANGUAGE_INSTRUCTION_LEFT_FIRST_CUBE]*NUM_EPISODE)
LANGUAGE_INSTRUCTIONS_RIGHT_FIRST_CUBE = np.array([LANGUAGE_INSTRUCTION_RIGHT_FIRST_CUBE]*NUM_EPISODE)

LANGUAGE_INSTRUCTION = b"pick up the cucumber and the red cube and put them in the bucket"
LANGUAGE_INSTRUCTIONS = np.array([LANGUAGE_INSTRUCTION]*NUM_EPISODE)

REWARDS = np.zeros(NUM_EPISODE, dtype=float)
REWARDS[-1] = 1
IS_TARMINALS = np.zeros(NUM_EPISODE, dtype=int)
IS_TARMINALS[-1] = 1
METADATA = sample_hdf5_file

In [8]:
def _image_bytes_feature(images):
    values = [tf.image.encode_jpeg(image).numpy() for image in images]
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=values))

def _language_bytes_feature(langs):
    values = [lang.tobytes() for lang in langs]
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=values))
    
def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))

def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))

In [9]:
def numpy_to_tf_example(states, images_top, images_angle, images_left_wrist, images_right_wrist, actions, lang=False, left_first_cucumber=False, right_first_cucumber=False, left_first_cube=False, right_first_cube=False):    
    if lang:   
        language_instructions = LANGUAGE_INSTRUCTIONS
    else:
        if left_first_cucumber:
            language_instructions = LANGUAGE_INSTRUCTIONS_LEFT_FIRST_CUCUMBER
        elif right_first_cucumber:
            language_instructions = LANGUAGE_INSTRUCTIONS_RIGHT_FIRST_CUCUMBER
        elif left_first_cube:
            language_instructions = LANGUAGE_INSTRUCTIONS_LEFT_FIRST_CUBE
        elif right_first_cube:
            language_instructions = LANGUAGE_INSTRUCTIONS_RIGHT_FIRST_CUBE

    feature = {
        "steps/is_first": _int64_feature(IS_FIRSTS),
        "steps/action": _float_feature(actions.flatten()),
        "steps/discount": _float_feature(DISCOUNTS),
        "steps/is_last": _int64_feature(IS_LASTS),
        "steps/language_instruction": _language_bytes_feature(language_instructions),
        "steps/rewards": _float_feature(REWARDS),
        "steps/observation/top": _image_bytes_feature(images_top),
        "steps/observation/angle": _image_bytes_feature(images_angle),
        "steps/observation/left_wrist": _image_bytes_feature(images_left_wrist),
        "steps/observation/right_wrist": _image_bytes_feature(images_right_wrist),
        "steps/is_terminal": _int64_feature(IS_TARMINALS),
        "steps/observation/state": _float_feature(states.flatten()),
        "episode_metadata/file_path": _bytes_feature(METADATA.encode())
    }

    features = tf.train.Features(feature=feature)
    example = tf.train.Example(features=features)
    
    return example

In [10]:
states_all = []
actions_all = []

In [11]:

episode_idx = 0
for file_idx in range(num_files):
    output_tfrecord_file = os.path.join(output_dir, output_basename.format(str(file_idx).zfill(5), str(num_files).zfill(5)))
    with tf.io.TFRecordWriter(output_tfrecord_file) as writer:
        for _ in range(episodes_per_file):
            hdf5_file = os.path.join(input_hdf5_dir, input_hdf5_basename.format(episode_idx))
            with h5py.File(hdf5_file, 'r') as hdf5_f:
                states = hdf5_f["observations"]["qpos"][:].copy()
                images_top = hdf5_f["observations"]["images"]["top"][:].copy()
                images_angle = hdf5_f["observations"]["images"]["angle"][:].copy()
                images_left_wrist = hdf5_f["observations"]["images"]["left_wrist"][:].copy()
                images_right_wrist = hdf5_f["observations"]["images"]["right_wrist"][:].copy()
                actions = hdf5_f["action"][:].copy()

                states_all.append(states)
                actions_all.append(actions)
            
            left_first_cucumber = episode_idx%4 == 0
            right_first_cucumber = episode_idx%4 == 1
            left_first_cube = episode_idx%4 == 2
            right_first_cube = episode_idx%4 == 3
                
            example = numpy_to_tf_example(states, images_top, images_angle, images_left_wrist, images_right_wrist, actions, 
                                          left_first_cucumber=left_first_cucumber, right_first_cucumber=right_first_cucumber, 
                                          left_first_cube=left_first_cube, right_first_cube=right_first_cube)   

            writer.write(example.SerializeToString())
            episode_idx += 1


2024-09-22 07:17:30.628695: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-09-22 07:17:30.684460: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-09-22 07:17:30.684564: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [12]:
states_all = np.concatenate(states_all)
actions_all = np.concatenate(actions_all)

In [13]:
statistics_dic = {}
statistics_dic["action"] = {}
statistics_dic["action"]["mean"] = actions_all.mean(axis=0).tolist()
statistics_dic["action"]["std"] = actions_all.std(axis=0).tolist()
statistics_dic["action"]["max"] = actions_all.max(axis=0).tolist()
statistics_dic["action"]["min"] = actions_all.min(axis=0).tolist()
statistics_dic["action"]["p99"] = np.percentile(actions_all, 99, axis=0).tolist()
statistics_dic["action"]["p01"] = np.percentile(actions_all, 1, axis=0).tolist()

statistics_dic["num_transitions"] = num_episodes * NUM_EPISODE
statistics_dic["num_trajectories"] = num_episodes

statistics_dic["proprio"] = {}
statistics_dic["proprio"]["mean"] = states_all.mean(axis=0).tolist()
statistics_dic["proprio"]["std"] = states_all.std(axis=0).tolist()
statistics_dic["proprio"]["max"] = states_all.max(axis=0).tolist()
statistics_dic["proprio"]["min"] = states_all.min(axis=0).tolist()
statistics_dic["proprio"]["p99"] = np.percentile(states_all, 99, axis=0).tolist()
statistics_dic["proprio"]["p01"] = np.percentile(states_all, 1, axis=0).tolist()


In [14]:
with open(os.path.join(output_dir, "dataset_statistics.json"), "w") as f:
    json.dump(statistics_dic, f)